In [1]:
api_ref = '''
共通

odpt:Calendar

曜日・日付区分

曜日・日付区分が記述される

共通

odpt:Operator

事業者

公共交通機関の事業者が記述される

鉄道

odpt:Station

駅情報

駅の名称や位置など、駅に関連する情報が記述される

鉄道

odpt:StationTimetable

駅時刻表

駅を発着する列車の時刻表情報が記述される

鉄道

odpt:Train

列車情報

列車の位置、行先など、列車のリアルタイムな情報が記述される

鉄道

odpt:TrainTimetable

列車時刻表

列車がどの駅にいつ到着するか、出発するかが記述される

鉄道

odpt:TrainInformation

運行情報

鉄道路線のリアルタイムな運行状況が記述される

鉄道

odpt:TrainType

列車種別

普通、快速など、列車の種別を定義する

鉄道

odpt:RailDirection

進行方向

上り、下りなど、列車の進行方向を定義する

鉄道

odpt:Railway

路線情報

鉄道における路線、運行系統が記述される

鉄道

odpt:RailwayFare

運賃情報

鉄道の運賃が記述される

鉄道

odpt:PassengerSurvey

駅別乗降人員

駅の乗降数集計結果が記述される

バス

odpt:Bus

バス情報

バス車両の位置、行先など、バス車両のリアルタイムな情報が記述される

バス

odpt:BusTimetable

バス時刻表

バスがあるバス停、バス標柱にいつ到着するか、いつ出発するかが記述される e.g. 系統時刻表、スターフ、運行表

バス

odpt:BusroutePattern

バス運行路線情報

バス運行における運行路線情報が記述される

バス

odpt:BusroutePatternFare

バス運賃情報

バスの運賃が記述される

バス

odpt:BusstopPole

バス停標柱情報

バス停における標柱情報が記述される

バス

odpt:BusstopPoleTimetable

バス停標柱時刻表

バスがあるバス停標柱にいつ到着するか、出発するかが記述される e.g. バス停時刻表、標柱時刻表、標柱

航空機

odpt:Airport

空港情報

空港の名称や位置など、空港に関連する情報が記述される

航空機

odpt:AirportTerminal

空港ターミナル情報

空港のターミナルの名称など、空港のターミナルに関連する情報が記述される

航空機

odpt:FlightInformationArrival

フライト到着情報

空港に当日到着する航空機のリアルタイムな情報が記述される

航空機

odpt:FlightInformationDeparture

フライト出発情報

空港を当日出発する航空機のリアルタイムな情報が記述される

航空機

odpt:FlightSchedule

フライト時刻表

航空機の予定される発着時刻情報が記述される e.g. 月間時刻表、週間スケジュール

航空機

odpt:FlightStatus

フライト状況

搭乗中、出発済みなど、空港を発着する航空機の状況を定義する
'''

In [2]:
import os, shutil
from datetime import datetime
import time
import json
import urllib.request, json 
from collections import Counter 
import random
import colorsys

import requests
from bs4 import BeautifulSoup

from tqdm import tqdm_notebook as tqdm

tr_dict = {
    "共通":"Common",
    "鉄道":"Train",
    "バス":"Bus",
    "航空機":"Aircraft"
}

s_ref = [ref for ref in api_ref.split('\n') if ref]

api_url = 'https://api-tokyochallenge.odpt.org/api/v4/'
api_url = 'https://api-tokyochallenge.odpt.org/api/v4/'
api_token = '620ebb7c8bc13f982014fc2e69e856644f045897a5343175e929d86a82013cdb'
api_endpoints = [{'type': tr_dict[s_ref[i]], 
                  'url':s_ref[i+1],
                  'real_time': 'リアルタイム' in s_ref[i+3]} for i in range(0, len(s_ref), 4)]

output_folder = 'C:/Users/umutk/Documents/Projects/odpt_2019/data/'

# Downloading data
---

In [ ]:
if os.path.exists(output_folder):
    shutil.rmtree(output_folder)
os.makedirs(output_folder, exist_ok=True)

with open(output_folder + "endpoints.json", 'w', encoding='utf8') as f:
    json.dump(api_endpoints, f)
    
for ep in tqdm(api_endpoints):
    if not ep['real_time']: # only download static data for now
        os.makedirs(f"{output_folder}/{ep['type']}", exist_ok=True)
        req_url = (api_url + ep['url'] + '?acl:consumerKey=' + api_token if ep['real_time'] 
                    else api_url + ep['url'] + '.json?acl:consumerKey=' + api_token)
                    
        with urllib.request.urlopen(req_url) as url:
            data = json.loads(url.read().decode())
            with open(f"{output_folder}/{ep['type']}/{ep['url'].replace(':', '_')}.json", 'w+', encoding='utf8') as f:
                json.dump(data, f, ensure_ascii=False)
        time.sleep(1)

In [ ]:
with open(f"{output_folder}Common/odpt_Calendar.json", encoding='utf8') as f:
    print(json.dumps(json.load(f), indent=4, sort_keys=True, ensure_ascii=False))

In [5]:
with open(f"{output_folder}Train/odpt_Railway.json", encoding='utf8') as f:
    railway = json.load(f)
with open(f"{output_folder}Train/odpt_Station.json", encoding='utf8') as f:
    station = json.load(f)
with open(f"{output_folder}Train/odpt_TrainTimetable.json", encoding='utf8') as f:
    train_time = json.load(f)

## Download color data
---

In [6]:
def scrape_web_color(url):
    cw = requests.get(url)
    soup = BeautifulSoup(cw.content)

    result = []
    for tr in soup.find_all("tr"):
        if tr.find_all("td")[0].has_attr('style'):
            result.append({'line_name': tr.find_all("td")[1].getText(), 
                           'line_color': tr.find_all("td")[0]["style"].split("; ")[0].split(": ")[-1].replace(";", "").upper()})
        else:
            result[-1]['line_name'] += (', ' + tr.find_all("td")[0].getText())
        
    return result

url_colors = "https://cdn.jsdelivr.net/npm/demo-dashi@0.2.1/src/line/tokyo.json"
with urllib.request.urlopen(url_colors) as url:
    line_colors_scrape = json.loads(url.read().decode())
line_colors_scrape = [l for g in line_colors_scrape['line_groups'] for l in g['lines']]

line_colors_scrape.extend(scrape_web_color("https://ayaito.net/webtips/color_code/12224/")) # kanto private railways
line_colors_scrape.extend(scrape_web_color("https://ayaito.net/webtips/color_code/115/")) # subways nationwide
line_colors_scrape.extend(scrape_web_color("https://ayaito.net/webtips/color_code/164/")) # JR group

# outlier line color additions
line_colors_scrape.extend([{'line_name': '中央線快速', 'line_color': '#f15a22'},
                            {'line_name': '中央・総武各駅停車', 'line_color': '#ffd400'},
                            {'line_name': '常磐線各駅停車', 'line_color': '#00b261'},
                            {'line_name': '常磐線快速', 'line_color': '#339999'},
                            {'line_name': '川越線(川越-高麗川間)', 'line_color': '#a8a39d'},
                            {'line_name': '南武線浜川崎支線', 'line_color': '#ffd600'},
                            {'line_name': '成田線我孫子支線', 'line_color': '#339999'},
                            {'line_name': '埼京線・川越線', 'line_color': '#00ac9a'},
                            {'line_name': '総武快速線', 'line_color': '#007ac0'},
                            {'line_name': '鶴見線大川支線', 'line_color': '#ffd400'},
                            {'line_name': '鶴見線海芝浦支線', 'line_color': '#ffd400'},
                            {'line_name': '日暮里・舎人ライナー', 'line_color': '#0099cc'},
                            {'line_name': '桐生線', 'line_color': '#FF0000'},
                            {'line_name': '小泉線(東小泉-太田)', 'line_color': '#FF0000'},
                            {'line_name': '小泉線', 'line_color': '#FF0000'},
                            {'line_name': '佐野線', 'line_color': '#FF0000'},
                            {'line_name': '東武スカイツリーライン(押上-曳舟)', 'line_color': '#0f6cc3'},
                            {'line_name': '丸ノ内線支線', 'line_color': '#f62e36'},
                            {'line_name': '会津線', 'line_color': '#006633'},
                            {'line_name': '秩父本線', 'line_color': '#ff6600'},
                            {'line_name': '芝山鉄道線', 'line_color': '#00a650'},
                            {'line_name': '新京成線', 'line_color': '#EF59A1'},
                            {'line_name': '会津鬼怒川線', 'line_color': '#ffa500 '},
                            {'line_name': '駿豆線', 'line_color': '#0000ff'},
                            {'line_name': 'ほくほく線', 'line_color': '#6633FF'},
                            {'line_name': '西武秩父線', 'line_color': '#ff6600'},
                            {'line_name': '西武園線', 'line_color': '#33cc66'},
                            {'line_name': '水郡線', 'line_color': '#368c44'}])

In [7]:
line_colors = {}

for r in railway:
    rw_name = r['dc:title']
    match = list(filter(lambda lc: lc['line_name'] in rw_name
                        or rw_name in lc['line_name'].replace(", ", 
                                                              "・").translate({ord(ch):'・' 
                                                                              for ch in '＜＞（）'}).split("・")
                        or rw_name == lc['line_name'].replace('東武鉄道', '').replace('東武', '').replace('東京メトロ','')
                        or rw_name.replace("線","鉄道") in lc['line_name']
                        or rw_name.replace("線","本線") in lc['line_name'], line_colors_scrape))
    
    if match:
        line_colors[r['owl:sameAs']] = match[0]['line_color'].upper()
len(line_colors), len(railway)

(181, 181)

In [8]:
with open(f"{output_folder}Train/line_colors.json", 'w+', encoding='utf8') as f:
    json.dump(line_colors, f, ensure_ascii=False)

In [9]:
def random_bright_color():
    h,s,l = random.random(), 0.5 + random.random()/2.0, 0.4 + random.random()/5.0
    r,g,b = [int(256*i) for i in colorsys.hls_to_rgb(h,l,s)]

    return f'#{r:02X}{g:02X}{b:02X}'

In [11]:
def get_line_color(r):
    return line_colors[r['owl:sameAs']]

In [12]:
with open(f"{output_folder}Train/line_colors.json", encoding='utf8') as f:
    line_colors = json.load(f)

# Parsing useful data
---

In [13]:
timetable = [{"code": tt["odpt:railway"],
              "t_number": tt["odpt:trainNumber"],
              "type": tt["odpt:trainType"],
              "timetable": [{'station':  (stt["odpt:departureStation"] 
                                           if "odpt:departureStation" in stt
                                           else stt["odpt:arrivalStation"]), 
                              'time': (stt["odpt:departureTime"] 
                                           if "odpt:departureTime" in stt
                                           else stt["odpt:arrivalTime"])}
                             for stt in tt["odpt:trainTimetableObject"]
                             if ("odpt:departureStation" in stt and "odpt:departureTime" in stt) 
                             or ("odpt:arrivalStation" in stt and "odpt:arrivalTime" in stt)]
             } for tt in train_time 
             if tt['odpt:calendar'] == 'odpt.Calendar:Weekday']

with open(f"{output_folder}Train/train_timetable_weekday.json", 'w+', encoding='utf8') as f:
    json.dump(timetable, f, ensure_ascii=False)

In [14]:
with open(f"{output_folder}Train/train_timetable_weekday.json", encoding='utf8') as f:
    timetable = json.load(f)

In [15]:
rail_station_times = [{"-".join(sorted([ts1['station'], ts2['station']])):
                       (datetime.strptime(ts2['time'], '%H:%M') -  datetime.strptime(ts1['time'], '%H:%M')).seconds}
                      for tt in timetable for ts1,ts2 in zip(tt['timetable'], tt['timetable'][1:])]
st_dist={}
for rs_pair in tqdm(rail_station_times):
    key, val = list(rs_pair.items())[0]
    st_dist[key] = st_dist.get(key, []) + [val] 
st_dist = {k : int(sum(v)/len(v)) for k, v in st_dist.items()}

In [16]:
line = []
for r in tqdm(railway):
    stations = []
    for idx, rs in enumerate(r['odpt:stationOrder']):
        st_info = next((s for s in station 
                        if rs['odpt:station'] == s["owl:sameAs"]
                        or (rs['odpt:station'].split('.')[-2:] == s["owl:sameAs"][-2:]
                            and r['owl:sameAs'] in s.get('odpt:connectingRailway', []))
                       ), None)
        if st_info and 'geo:lat' in st_info:
            passing_trains = st_info.get('odpt:connectingRailway', []) + [st_info['odpt:railway']]
            stations.append({
                'idx': rs['odpt:index'], 
                'code': rs['odpt:station'],
                 # 'st_code': st_info['odpt:stationCode'] if 'odpt:stationCode' in st_info else '',
                'title': st_info['odpt:stationTitle']['en'], 
                'dur': 0 if rs['odpt:station'] == r['odpt:stationOrder'][-1]['odpt:station'] 
                else st_dist.get('-'.join(sorted([rs['odpt:station'], r['odpt:stationOrder'][idx+1]['odpt:station']])), 180),
                # 'title_ja': st_info['odpt:stationTitle']['ja'],
                'geo': {'lat': st_info['geo:lat'], 'long': st_info['geo:long']},
                'n_trains': len(passing_trains)
            })
    stations = sorted(stations, key=lambda k: k['idx'])

    if len(stations)>2:
        line.append({
            'code': r['owl:sameAs'], 
            'title': r['odpt:railwayTitle']['en'], 
             # 'title_ja': r['odpt:railwayTitle']['ja'],
             # 'repr_color': get_line_color(r),
            'station': stations
        })

In [17]:
# Save station coordinates for lines
line = [l for l in line if 'station' in l and len(l['station']) > 1]
with open(f"{output_folder}Train/line_station_coords.json", 'w+', encoding='utf8') as f:
                json.dump(line, f, ensure_ascii=False)

In [18]:
with open(f"{output_folder}Train/line_station_coords.json", encoding='utf8') as f:
    line = json.load(f)
    print(json.dumps(line[0], indent=4, sort_keys=True, ensure_ascii=False))

{
    "code": "odpt.Railway:Odakyu.Tama",
    "station": [
        {
            "code": "odpt.Station:Odakyu.Tama.ShinYurigaoka",
            "dur": 180,
            "geo": {
                "lat": 35.60384,
                "long": 139.50772
            },
            "idx": 1,
            "n_trains": 2,
            "title": "Shin-Yurigaoka"
        },
        {
            "code": "odpt.Station:Odakyu.Tama.Satsukidai",
            "dur": 180,
            "geo": {
                "lat": 35.60014,
                "long": 139.49365
            },
            "idx": 2,
            "n_trains": 1,
            "title": "Satsukidai"
        },
        {
            "code": "odpt.Station:Odakyu.Tama.Kurihira",
            "dur": 180,
            "geo": {
                "lat": 35.60599,
                "long": 139.4809
            },
            "idx": 3,
            "n_trains": 1,
            "title": "Kurihira"
        },
        {
            "code": "odpt.Station:Odakyu.Tama.Kurokawa",
  